In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Rescaling, Resizing
from tensorflow.keras.models import Model

# Load the pre-trained model
saved_model_path = '/home/athena/Documents/GitHub/Dog-NosePrint-Recognition/keras_model'
loaded_model = tf.keras.models.load_model(saved_model_path)


# Define the initial input shape for raw images (this can be any size)
input_shape = (256, 256, 3)  # None dimensions allow for variable image sizes
inputs = Input(shape=input_shape, dtype='uint8')

x = tf.cast(inputs, tf.float32)

x = Rescaling(1./255)(x)        # Normalizes the pixel values to [0, 1]

# Feed the preprocessed input into the loaded model
outputs = loaded_model(x)

# Create the new model with preprocessing layers
new_model = Model(inputs=inputs, outputs=outputs)

# Print the new model summary
new_model.summary()


2024-07-25 20:40:53.606726: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-25 20:40:53.643791: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 20:40:53.643827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 20:40:53.644977: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 20:40:53.650731: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-25 20:40:53.651435: I tensorflow/core/platform/cpu_feature_guard.cc:1

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 tf.cast (TFOpLambda)        (None, 256, 256, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 model (Functional)          (1, 1024)                 84257730  
                                                                 
Total params: 84257730 (321.42 MB)
Trainable params: 84103234 (320.83 MB)
Non-trainable params: 154496 (603.50 KB)
_________________________________________________________________


In [3]:
import numpy as np
from PIL import Image

# Function to load the image using NumPy
def load_raw_image(image_path):
    img = Image.open(image_path)  # Load image
    img = img.resize((256, 256), Image.BILINEAR)
    img_array = np.array(img)  # Convert to NumPy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Perform inference
def predict(model, img_path):
    img_array = load_raw_image(img_path)
    predictions = model.predict(img_array)
    return predictions

# Path to the image and the enhanced model
img_path = 'image.jpg'

# Get predictions
image = load_raw_image(img_path)
predictions = new_model.predict(image)

# Output the predictions
print(predictions)


1/1 [==============================] - 1s 1s/step
[[ -6.588468  -13.785604  -12.895827  ...  -1.7320358   9.248583
   -7.9605794]]


In [4]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
tflite_model = converter.convert()

# Save the TFLite model to a file
tflite_model_path = 'dog_nose.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved to {tflite_model_path}")

INFO:tensorflow:Assets written to: /tmp/tmplk4r1vod/assets


INFO:tensorflow:Assets written to: /tmp/tmplk4r1vod/assets
2024-07-25 20:43:01.358676: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-25 20:43:01.358720: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-25 20:43:01.359339: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmplk4r1vod
2024-07-25 20:43:01.449259: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-25 20:43:01.449359: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmplk4r1vod
2024-07-25 20:43:01.625284: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-07-25 20:43:01.670099: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-25 20:43:02.787752: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

TFLite model saved to dog_nose.tflite


In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_fp16_model = converter.convert()
tflite_model_fp16_file = "dog_nose_quant_f16.tflite"

INFO:tensorflow:Assets written to: /tmp/tmpibq3lqie/assets


INFO:tensorflow:Assets written to: /tmp/tmpibq3lqie/assets
2024-07-25 20:51:17.051968: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-25 20:51:17.052011: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-25 20:51:17.052212: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpibq3lqie
2024-07-25 20:51:17.144601: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-25 20:51:17.144669: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpibq3lqie
2024-07-25 20:51:17.405129: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-25 20:51:18.629179: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpibq3lqie
2024-07-25 20:51:19.060211: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

AttributeError: 'str' object has no attribute 'write_bytes'

In [7]:
with open(tflite_model_fp16_file, 'wb') as f:
    f.write(tflite_fp16_model)

In [5]:
import numpy as np
import tensorflow as tf
from PIL import Image

def load_raw_image(image_path):
    img = Image.open(image_path)  # Load image
    img = img.resize((256, 256), Image.BILINEAR)
    img_array = np.array(img)  # Convert to NumPy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='dog_nose.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Details:", input_details)
print("Output Details:", output_details)
def predict(image_path):
    img_array = load_raw_image(image_path)

    # Set the tensor to point to the input data to be inferred
    interpreter.set_tensor(input_details[0]['index'], img_array)
    interpreter.invoke()

    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

# Path to the image
img_path = 'image.jpg'

# Perform inference and get predictions
predictions = predict(img_path)
print("Predictions:", predictions)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Input Details: [{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([  1, 256, 256,   3], dtype=int32), 'shape_signature': array([ -1, 256, 256,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output Details: [{'name': 'StatefulPartitionedCall:0', 'index': 627, 'shape': array([   1, 1024], dtype=int32), 'shape_signature': array([   1, 1024], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Predictions: [[ -6.588463  -13.785602  -12.895834  ...  -1.7320392   9.248584
   -7.9605727]]
